# `lastScore` Results Data

Accessing stage and overal results and timing data.

This is probably the data you're most interested in, so let's get stuck into it.

The data is defined at the category and stage level and includes stages results and timing data at each lastscore, as well as at stage end, as well as overall timing and results classification data.

In [1]:
# Load in the required packages
import pandas as pd
from jupyterlite_simple_cors_proxy import furl, xurl

# Generate the API URL pattern
dakar_api_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/{path}"

# Define the year
YEAR = 2025
# Define the category
CATEGORY = "A"
# Define the stage
STAGE = 1

# Define the API path to the stage resource
# Use a Python f-string to instantiate variable values directly
lastscore_path = f"lastScore-{YEAR}-{CATEGORY}-{STAGE}"

# Define the URL
lastscore_url = dakar_api_template.format(path=lastscore_path)

# Preview the path and the URL
lastscore_path, lastscore_url

('lastScore-2025-A-1',
 'https://www.dakar.live.worldrallyraidchampionship.com/api/lastScore-2025-A-1')

The JSON object returned by the live data service for the `lastScore` results feeds is quite a deepy nested data structure that returns a complex dataframe is parsed directly using the `read_json()` function:

In [ ]:
# Load in data
# Use furl() to handle CORS issues in Jupyterlite
_lastscore_df = pd.read_json(furl(lastscore_url))
_lastscore_df.head(2)

,team,dss,bonif,cg,cs,ce,fsh,wd,_bind,_updatedAt,_id
0,"{'bib': 427, 'clazz': '9a68ed3c41c5c7a1642df5d...","{'position': 130, 'absolute': 40650000, 'inter...","{'total': 0, 'wp': []}","{'01216': {'position': [128, 128], 'absolute':...","{'01207': {'position': [129, 129], 'absolute':...","{'position': [125, 125], 'absolute': [23353000...",True,False,lastScore-2025-A-1,1736257655156,lastScore-2025-A-1-427
1,"{'bib': 634, 'clazz': 'ec2f26ebeb14824160c7204...","{'position': 171, 'absolute': 41910000, 'inter...","{'total': 0, 'wp': []}","{'01216': {'position': [194, 194], 'absolute':...","{'01207': {'position': [179, 179], 'absolute':...","{'position': [196, 197], 'absolute': [11880000...",False,True,lastScore-2025-A-1,1736257727912,lastScore-2025-A-1-634


We can get a much better behaved data structure if we parse the JSON directly using the `pd.json_normalize()` function:

In [ ]:
# Rather than re-download the JSON data using requests,
# (remembering to use xurl() for JupyterLite CORS handling)
# we can convert the dataframe back to JSON
_lastscore_json = _lastscore_df.to_dict('records')

# Now we can use the more powerful json_normalize() function
# to generate a dataframe from the data
_results_df = pd.json_normalize(_lastscore_json)

_results_df.head()

,fsh,wd,_bind,_updatedAt,_id,team.bib,team.clazz,team.brand,team.model,team.competitors,...,cs.01ASS.absolute,cs.01ASS.relative,cs.01ASS.bonus,ce.position,ce.absolute,ce.relative,ce.bonus,cg.01ASS.penality,cs.01ASS.penality,ce.penality
0,True,False,lastScore-2025-A-1,1736257655156,lastScore-2025-A-1-427,427,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'B. LEPIETRE', 'firstName': 'BENOIT'...",...,"[23353000, 23353000]","[6845000, 6845000]",0,"[125, 125]","[23353000, 23353000]","[6845000, 6845000]",0,NaN,NaN,NaN
1,False,True,lastScore-2025-A-1,1736257727912,lastScore-2025-A-1-634,634,ec2f26ebeb14824160c7204618a5780d,DAF,FAV 85 MX,"[{'name': 'J. ESTEVE ORO', 'firstName': 'JORDI...",...,"[118800000, 118800000]","[102292000, 102292000]",0,"[196, 197]","[118800000, 118800000]","[102292000, 102292000]",0,79200000.0,79200000.0,79200000.0
2,True,False,lastScore-2025-A-1,1736257728757,lastScore-2025-A-1-330,330,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,"[{'name': 'A. ALKUWARI', 'firstName': 'AHMED F...",...,"[18180000, 18180000]","[1672000, 1672000]",0,"[43, 43]","[18180000, 18180000]","[1672000, 1672000]",0,NaN,NaN,NaN
3,True,False,lastScore-2025-A-1,1736257728636,lastScore-2025-A-1-243,243,f00d7ec8d2d96e9cf11aa515109376cf,MD,OPTIMUS,"[{'name': 'P. THOMASSE', 'firstName': 'PASCAL'...",...,"[18668000, 18668000]","[2160000, 2160000]",0,"[50, 50]","[18668000, 18668000]","[2160000, 2160000]",0,NaN,NaN,NaN
4,True,False,lastScore-2025-A-1,1736257724233,lastScore-2025-A-1-404,404,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'F. LOPEZ CONTARDO', 'firstName': 'F...",...,"[18018000, 18018000]","[1510000, 1510000]",0,"[34, 34]","[18018000, 18018000]","[1510000, 1510000]",0,NaN,NaN,NaN


Inspection of the dataframe column names shows how the `.json_normalize()` function flattened nested objects into and created flattened, dotted column names that reveal the original nested object data paths.

In [14]:
_results_df.columns

Index(['fsh', 'wd', '_bind', '_updatedAt', '_id', 'team.bib', 'team.clazz',
       'team.brand', 'team.model', 'team.competitors', 'team.vehicle',
       'team.vehicleImg', 'team.w2rc', 'dss.position', 'dss.absolute',
       'dss.real', 'bonif.total', 'bonif.wp', 'cg.01216.position',
       'cg.01216.absolute', 'cg.01216.relative', 'cg.01218.position',
       'cg.01218.absolute', 'cg.01218.relative', 'cg.01224.position',
       'cg.01224.absolute', 'cg.01224.relative', 'cg.01207.position',
       'cg.01207.absolute', 'cg.01207.relative', 'cg.01233.position',
       'cg.01233.absolute', 'cg.01233.relative', 'cg.01230.position',
       'cg.01230.absolute', 'cg.01230.relative', 'cg.01222.position',
       'cg.01222.absolute', 'cg.01222.relative', 'cg.01220.position',
       'cg.01220.absolute', 'cg.01220.relative', 'cg.01ASS.position',
       'cg.01ASS.absolute', 'cg.01ASS.relative', 'cg.01ASS.bonus',
       'cg.01ASS.stagePenalty', 'cg.01227.position', 'cg.01227.absolute',
       'cg.012

We might also notice that as well as the `ce`, `cg` and `cs` results fields, which are annoteated with what we might assume to be waypoint identifiers, the dataframe includes `team` and crew information which is not directly associated with a particular stage, but is rather more generic.

It therefore makes sense to extract this data from the data frame, so that we might store it separately (we might also assume that the crew and team information will be fixed throughout the rally? Or are crew changes allowed during the rally?)

The following function

In [25]:
import pandas as pd
from typing import Tuple


def normalize_team_competitors(df: pd.DataFrame, year: int = 2025) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Transform a DataFrame containing nested competitor lists into three slighly more normalized DataFrames.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns including 'team.bib', 'team.model', and 'team.competitors' (list of dicts)
    
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: (teams_df, competitors_df, results_df)
            - teams_df: DataFrame with team information
            - competitors_df: DataFrame with competitor information
            - results_df: DataFrame of results;
    """
    # Create teams DataFrame by dropping the competitors column
    teams_df = df.drop('team.competitors', axis=1)

    # Create competitors DataFrame using explode and json_normalize
    competitors_df = (
        df[['team.bib', 'team.competitors']]
        .explode('team.competitors')
        .rename(columns={'team.bib': 'team_bib'})
        .reset_index(drop=True)
    )

    # Normalize the dictionary contents and combine with team_bib
    competitors_df = pd.concat([
        competitors_df['team_bib'],
        pd.json_normalize(competitors_df['team.competitors'])
    ], axis=1)

    # Align column names with an ealier Dakar analysis codebase
    competitors_df["Year"] = year
    competitors_df.rename(
        columns={'team.bib': 'Bib', 'name': 'Name'}, inplace=True)

    team_cols = [c for c in teams_df.columns if c.startswith(
        "team")]
    teams_df = teams_df[team_cols]
    teams_df.rename(columns={'team.bib': 'Bib'}, inplace=True)

    team_cols.remove("team.bib")
    team_cols.append("team.competitors")
    return teams_df, competitors_df, df.rename(columns={'team.bib': 'Bib'}).drop(team_cols, axis=1)

Applying this function splits out the team, competitor, and results data.

In [27]:
teams_df, competitors_df, _results = normalize_team_competitors(_results_df)

In [16]:
# Preview the team data
teams_df.head()

,Bib,team.clazz,team.brand,team.model,team.vehicle,team.vehicleImg,team.w2rc
0,427,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,BTR,https://img.aso.fr/core_app/img-motorSports-da...,False
1,634,ec2f26ebeb14824160c7204618a5780d,DAF,FAV 85 MX,TIBAU TEAM,https://img.aso.fr/core_app/img-motorSports-da...,False
2,330,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,NASSER RACING,https://img.aso.fr/core_app/img-motorSports-da...,False
3,243,f00d7ec8d2d96e9cf11aa515109376cf,MD,OPTIMUS,MD RALLYE SPORT,https://img.aso.fr/core_app/img-motorSports-da...,False
4,404,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,CAN-AM FACTORY TEAM,https://img.aso.fr/core_app/img-motorSports-da...,False


In [17]:
# Preview the competitors data
competitors_df.head()

,team_bib,Name,firstName,lastName,role,gender,nationality,profil,profil_sm,podium,aid,Year
0,427,B. LEPIETRE,BENOIT,LEPIETRE,P,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,110add16-df63-4b90-9494-bc3db7d02662,2025
1,427,R. RELMY-MADINSKA,RODRIGUE,RELMY-MADINSKA,C,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,63cb271c-1c74-4f07-82b4-c3df3c425aa7,2025
2,634,J. ESTEVE ORO,JORDI,ESTEVE ORO,P,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,36106f23-d545-47b9-8565-983cce7550d3,2025
3,634,F. PARDO,FRANCESC,PARDO,C,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,43c1ce16-ccee-4ac2-ae16-b89e82a43183,2025
4,634,J. PUJOL FORNOS,JORDI,PUJOL FORNOS,M,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,7700b503-68b6-4aca-a63a-96047071c2e9,2025


In [18]:
# Preview the results data
_results.head()

,fsh,wd,_bind,_updatedAt,_id,Bib,dss.position,dss.absolute,dss.real,bonif.total,...,cs.01ASS.absolute,cs.01ASS.relative,cs.01ASS.bonus,ce.position,ce.absolute,ce.relative,ce.bonus,cg.01ASS.penality,cs.01ASS.penality,ce.penality
0,True,False,lastScore-2025-A-1,1736257655156,lastScore-2025-A-1-427,427,130,40650000,True,0,...,"[23353000, 23353000]","[6845000, 6845000]",0,"[125, 125]","[23353000, 23353000]","[6845000, 6845000]",0,NaN,NaN,NaN
1,False,True,lastScore-2025-A-1,1736257727912,lastScore-2025-A-1-634,634,171,41910000,True,0,...,"[118800000, 118800000]","[102292000, 102292000]",0,"[196, 197]","[118800000, 118800000]","[102292000, 102292000]",0,79200000.0,79200000.0,79200000.0
2,True,False,lastScore-2025-A-1,1736257728757,lastScore-2025-A-1-330,330,55,38370000,True,0,...,"[18180000, 18180000]","[1672000, 1672000]",0,"[43, 43]","[18180000, 18180000]","[1672000, 1672000]",0,NaN,NaN,NaN
3,True,False,lastScore-2025-A-1,1736257728636,lastScore-2025-A-1-243,243,67,38730000,True,0,...,"[18668000, 18668000]","[2160000, 2160000]",0,"[50, 50]","[18668000, 18668000]","[2160000, 2160000]",0,NaN,NaN,NaN
4,True,False,lastScore-2025-A-1,1736257724233,lastScore-2025-A-1-404,404,72,38880000,True,0,...,"[18018000, 18018000]","[1510000, 1510000]",0,"[34, 34]","[18018000, 18018000]","[1510000, 1510000]",0,NaN,NaN,NaN


The results dataframe still looks rather cluttered. Inspection of the data, and comparison back with the parent webiste, sugghests the following interpretation for some of the results data columns:

- `ce`: end of stage status
- `cg`: overall status at waypoint
- `cs`: stage status at waypoint
- `dss`: stage start status

Ideally, we would like to be able retrieve different collections of results data in a natural way.

For example, we might want to query:

- the end of stage positions for each crew;
- the positions of each crew at a particular waypoint;
- time differences or gap to leader at stage end, or at a particular waypoint;
- the position of a particular crew at each waypoint.

Having the data in a particular form can often make it easier — or harder — to make these sorts of query.


In [23]:
id_column = "_id"
point_cols = [col for col in _results.columns if col.startswith(('cg', 'cs'))]
# Melt only the point-specific columns
melted = _results[[id_column, "Bib", *point_cols]
                  ].melt(id_vars=[id_column, "Bib"]).dropna()
melted = melted[melted['variable'].str.contains('position|absolute|relative')]
melted["type"] = melted["variable"].str.split('.').str[0]
melted["waypoint"] = melted["variable"].str.extract(r'\.([^\.]+)\.')
melted["metric"] = melted["variable"].str.split('.').str[-1]

melted[['value_0', 'value_1']] = pd.DataFrame(
    melted['value'].tolist(),
    index=melted.index
)
# TO DO - drop: variable, value, _id
# TO DO - have a backup table of _id and Bib
# TO DO - add: Year, Stage
melted

,_id,Bib,variable,value,type,waypoint,metric,value_0,value_1
0,lastScore-2025-A-1-427,427,cg.01216.position,"[128, 128]",cg,01216,position,128,128
1,lastScore-2025-A-1-634,634,cg.01216.position,"[194, 194]",cg,01216,position,194,194
2,lastScore-2025-A-1-330,330,cg.01216.position,"[53, 53]",cg,01216,position,53,53
3,lastScore-2025-A-1-243,243,cg.01216.position,"[60, 60]",cg,01216,position,60,60
4,lastScore-2025-A-1-404,404,cg.01216.position,"[65, 65]",cg,01216,position,65,65
...,...,...,...,...,...,...,...,...,...
12457,lastScore-2025-A-1-212,212,cs.01ASS.relative,"[133000, 133000]",cs,01ASS,relative,133000,133000
12458,lastScore-2025-A-1-208,208,cs.01ASS.relative,"[1662000, 1662000]",cs,01ASS,relative,1662000,1662000
12459,lastScore-2025-A-1-313,313,cs.01ASS.relative,"[5299000, 5299000]",cs,01ASS,relative,5299000,5299000
12460,lastScore-2025-A-1-628,628,cs.01ASS.relative,"[15065000, 15065000]",cs,01ASS,relative,15065000,15065000
